IMPORT DATA DAN ZIP

In [ ]:
import pandas as pd
import xml.etree.ElementTree as xet

In [ ]:
from glob import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
Test = ("/content/drive/MyDrive/DataTest.zip")

with ZipFile(Test, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import pandas as pd
import cv2
df = pd.read_csv('/content/drive/MyDrive/datatrainfix.csv')
df

,Id,Vehicleregistrationplate,NameofFile
0,1,A7814,DataTrain1.png
1,2,B1074QO,DataTrain2.png
2,3,B1031QO,DataTrain3.png
3,4,B187EDA,DataTrain4.png
4,5,B1089VD,DataTrain5.png
...,...,...,...
795,796,B1677EJC,DataTrain796.png
796,797,B1743VO,DataTrain797.png
797,798,AD1416YD,DataTrain798.png
798,799,AB5419TN,DataTrain799.png


In [ ]:
from zipfile import ZipFile
Train = ("/content/drive/MyDrive/DataTrain.zip")

with ZipFile(Train, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
from zipfile import ZipFile
label = ("/content/drive/MyDrive/labels.zip")

with ZipFile(label, 'r') as zip:
  zip.extractall()
  print('Done')

Done


SPLIT DATA

In [ ]:
import os
import random
import shutil
input_folder = ("/content/Data Train")
train_folder = ("/content/databaru/train/images")
val_folder = ("/content/databaru/test/images")
split_ratio = 0.8  # Rasio pembagian data (misalnya 0.8 berarti 80% set latihan, 20% set validasi)

# Mendapatkan daftar file dalam folder input
file_list = os.listdir(input_folder)
random.shuffle(file_list)  # Mengacak urutan file

# Menghitung jumlah file untuk setiap set
num_files = len(file_list)
num_train = int(num_files * split_ratio)
num_val = num_files - num_train

# Membuat folder set latihan dan set validasi jika belum ada
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Memindahkan file ke set latihan
for file_name in file_list[:num_train]:
    src_path = os.path.join(input_folder, file_name)
    dst_path = os.path.join(train_folder, file_name)
    shutil.copy(src_path, dst_path)

# Memindahkan file ke set validasi
for file_name in file_list[num_train:]:
    src_path = os.path.join(input_folder, file_name)
    dst_path = os.path.join(val_folder, file_name)
    shutil.copy(src_path, dst_path)


In [ ]:
import os
import random
import shutil
input_labels = ("/content/labels")
train_labels = ("/content/databaru/train/labels")
val_labels = ("/content/databaru/test/labels")
split_ratio = 0.8  # Rasio pembagian data (misalnya 0.8 berarti 80% set latihan, 20% set validasi)

# Mendapatkan daftar file dalam folder input
file_list = os.listdir(input_labels)
random.shuffle(file_list)  # Mengacak urutan file

# Menghitung jumlah file untuk setiap set
num_files = len(file_list)
num_train = int(num_files * split_ratio)
num_val = num_files - num_train

# Membuat folder set latihan dan set validasi jika belum ada
os.makedirs(train_labels, exist_ok=True)
os.makedirs(val_labels, exist_ok=True)

# Memindahkan file ke set latihan
for file_name in file_list[:num_train]:
    src_path = os.path.join(input_labels, file_name)
    dst_path = os.path.join(train_labels, file_name)
    shutil.copy(src_path, dst_path)

# Memindahkan file ke set validasi
for file_name in file_list[num_train:]:
    src_path = os.path.join(input_labels, file_name)
    dst_path = os.path.join(val_labels, file_name)
    shutil.copy(src_path, dst_path)

train yolov8

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15814, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15814 (delta 9), reused 23 (delta 2), pack-reused 15768
Receiving objects: 100% (15814/15814), 14.64 MiB | 9.58 MiB/s, done.
Resolving deltas: 100% (10821/10821), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 625.9/625.9 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
import yaml
data = dict(
    train= ('/content/databaru/train/images'),
    val= ('/content/databaru/test/images'),
    nc = 36,
    names = ['A','B','C','D','E','F','G','H','I','J','K',
             'L','M','N','O','P','Q','R','S','T','U','V','W','X','Y',
             'Z','0','1','2','3','4','5','6','7','8','9']
)

# Mengonversi data ke format YAML
yaml_content = yaml.dump(data)

# Menyimpan konten YAML ke dalam file
with open('data.yaml', 'w') as file:
    file.write(yaml_content)

print("File YAML telah berhasil dibuat.")

File YAML telah berhasil dibuat.


In [ ]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()

In [ ]:
!python train.py --data data.yaml --cfg models/yolov5s.yaml --batch-size 64 --name Model --epochs 300

In [ ]:
## TO PREDICT IMAGES IN A FOLDER ##
!python detect.py --source '/content/DataTest/Data Test for BDC 2023 - Penyisihan' --weights '/content/drive/MyDrive/runs/train/Model/weights/best.pt' --save-txt

detect: weights=['/content/drive/MyDrive/runs/train/Model/weights/best.pt'], source=/content/DataTest/Data Test for BDC 2023 - Penyisihan, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-193-g485da42 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7107217 parameters, 0 gradients, 16.1 GFLOPs
image 1/100 /content/DataTest/Data Test for BDC 2023 - Penyisihan/DataTest1.png: 224x640 1 A, 1 D, 1 E, 1 O, 2 0s, 1 3, 1 4, 1 7, 39.6ms
image 2/100 /content/DataTest/Data Test for BDC 2023 - Penyisihan/DataTest10.png: 256x640 2 Bs, 1 H, 1 Y, 1 1, 1 3, 1 6, 1 7, 56.3ms
image 3/100 /con

In [ ]:
import torch
import pandas as pd

def process_image(DataTest):
    model = torch.hub.load('ultralytics/yolov5', 'custom', path="/content/drive/MyDrive/runs/train/Model/weights/best.pt")
    results = model(DataTest)
    results.print()

    pd_box = results.pandas().xyxy[0]
    pd_box = pd_box.sort_values('xmin')

    pd_box['selisih'] = pd_box['xmax'] - pd_box['xmin']
    s_rata = pd_box['selisih'].mean()

    s = []
    for i in range(1, len(pd_box)):
        kurang = pd_box['xmax'].iloc[i-1] - pd_box['xmin'].iloc[i]
        s.append(kurang)
    pd_box['s'] = [None] + s

    baris = len(pd_box)
    baris_dihapus = []

    for i in range(1, baris):
        if pd_box['s'].iloc[i] <= 0:
            continue
        elif pd_box['s'].iloc[i] > 0:
            cek = pd_box['s'].iloc[i] / s_rata
            if cek >= 0.5:
                if pd_box['confidence'].iloc[i] <= pd_box['confidence'].iloc[i-1]:
                     baris_dihapus.append(i)
                elif pd_box['confidence'].iloc[i] > pd_box['confidence'].iloc[i-1]:
                    baris_dihapus.append(i-1)
            elif cek < 0.5:
                continue

    pd_box = pd_box.drop(pd_box.index[baris_dihapus])
    return pd_box['name'].tolist()

# Membuat dictionary dengan key berupa nama gambar dan value berupa hasil looping
result_dict = {}
for i in range(1, 101):
    image_path = f'/content/DataTest/Penyisihan/DataTest{i}.png'  # Ganti dengan path dan format nama gambar yang sesuai
    processed_data = process_image(image_path)
    result_dict[f'DataTest{i}'] = processed_data

# Membuat DataFrame dari dictionary
result_df = pd.DataFrame(result_dict.items(), columns=['File Gambar', 'List Name'])

print(result_df)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-7 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7107217 parameters, 0 gradients, 16.1 GFLOPs
Adding AutoShape... 
image 1/1: 112x358 1 A, 1 D, 1 E, 1 O, 2 0s, 1 3, 1 4, 1 7
Speed: 2.9ms pre-process, 6.1ms inference, 1.4ms NMS per image at shape (1, 3, 224, 640)
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-7 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7107217 parameters, 0 gradients, 16.1 GFLOPs
Adding AutoShape... 
image 1/1: 98x318 1 A, 1 E, 1 X, 1 3, 2 8s, 1 9
Speed: 2.4ms pre-process, 6.2ms inference, 1.3ms NMS per image at shape (1, 3, 224, 640)
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-7 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 

    File Gambar                 List Name
0     DataTest1  [A, D, 7, 0, 3, 4, O, E]
1     DataTest2     [A, 9, 3, 8, 8, E, X]
2     DataTest3                 [B, 1, T]
3     DataTest4  [B, 1, 6, 6, 1, T, K, Z]
4     DataTest5  [A, D, 3, 7, 7, 2, 4, B]
..          ...                       ...
95   DataTest96           [1, 2, 8, 5, U]
96   DataTest97  [A, B, 8, 6, 4, 4, P, K]
97   DataTest98                 [A, E, G]
98   DataTest99     [B, 1, 5, 0, 9, U, N]
99  DataTest100     [B, 1, 4, 0, 8, R, X]

[100 rows x 2 columns]


In [ ]:
# Mengkonversi float menjadi list
result_df['List Name'] = result_df['List Name'].apply(lambda x: x if isinstance(x, list) else [x])

# Menghapus tanda kutip dan tanda koma, dan menyisakan karakter saja
result_df['List Name'] = result_df['List Name'].apply(lambda x: [str(name).replace('"', '').strip() for name in x])

# Menyimpan DataFrame ke dalam file CSV
file_path = '/content/submissiomlagi.csv'  # Path file CSV yang akan disimpan
result_df.to_csv(file_path, index=False)

# Menampilkan nama file hasil.csv
print("File CSV telah disimpan sebagai hasil.csv")

File CSV telah disimpan sebagai hasil.csv


In [ ]:
# Mengkonversi float menjadi list
result_df['List Name'] = result_df['List Name'].apply(lambda x: x if isinstance(x, list) else [x])

# Menghapus tanda kutip dan tanda koma, dan menyisakan karakter saja
result_df['List Name'] = result_df['List Name'].apply(lambda x: [str(name).replace('"', '').strip() for name in x])

# Menampilkan nama file hasil.csv
result_df

,File Gambar,List Name
0,DataTest1,"[A, D, 7, 0, 3, 4, O, E]"
1,DataTest2,"[A, 9, 3, 8, 8, E, X]"
2,DataTest3,"[B, 1, T]"
3,DataTest4,"[B, 1, 6, 6, 1, T, K, Z]"
4,DataTest5,"[A, D, 3, 7, 7, 2, 4, B]"
...,...,...
95,DataTest96,"[1, 2, 8, 5, U]"
96,DataTest97,"[A, B, 8, 6, 4, 4, P, K]"
97,DataTest98,"[A, E, G]"
98,DataTest99,"[B, 1, 5, 0, 9, U, N]"


In [ ]:
#menyatukan kolom list name
result_df['List Name'] = result_df['List Name'].astype(str).str.replace(' ', '').str.replace('[', '').str.replace(',', '').str.replace("'", '').str.replace(']', '')
#menambahkan kolom nomor
nomor = list(range(100))
result_df.insert(0,'', nomor)
#menambahkan .png tiap baris file gambar
result_df['File Gambar'] = result_df['File Gambar'] + '.png'
#rename nama kolom sesuai format
result_df = result_df.rename(columns={'File Gambar' : 'Name of File', 'List Name': 'Vehicleregistrationplatebymodel'})
#menampilkan dataframe
result_df

<ipython-input-42-650908dd0148>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result_df['List Name'] = result_df['List Name'].astype(str).str.replace(' ', '').str.replace('[', '').str.replace(',', '').str.replace("'", '').str.replace(']', '')


,,Name of File,Vehicleregistrationplatebymodel
0,0,DataTest1.png,AD7034OE
1,1,DataTest2.png,A9388EX
2,2,DataTest3.png,B1T
3,3,DataTest4.png,B1661TKZ
4,4,DataTest5.png,AD37724B
...,...,...,...
95,95,DataTest96.png,1285U
96,96,DataTest97.png,AB8644PK
97,97,DataTest98.png,AEG
98,98,DataTest99.png,B1509UN


In [ ]:
# Menyimpan DataFrame ke dalam file CSV
file_path = '/content/submissionlago.csv'  # Path file CSV yang akan disimpan
result_df.to_csv(file_path, index=False)

In [ ]:
def akurasi(test, prediksi):
  tt = test.ljust(9)
  pp = prediksi.ljust(9)
  r = [1 if x==y else 0 for (x,y) in zip(tt,pp)]
  s = sum(r)
  return s/9

In [ ]:
dataplat = pd.read_csv("/content/drive/MyDrive/dataplat.csv")
dataplat['Prediksi'] = result_df['Vehicleregistrationplatebymodel']
file_path = '/content/datayolov5baru.csv'
dataplat.to_csv(file_path, index=False)

In [ ]:
for i in range(0, 100):
  hasil_akurasi = akurasi(dataplat['Plat Sebenarnya'].iloc[i], dataplat['Prediksi'].iloc[i])
hasil_akurasi

TypeError: ignored

In [ ]:
total = 0
for i in range(0, 100):
  hasil_akurasi = akurasi(dataplat['Plat Sebenarnya'].iloc[i], dataplat['Prediksi'].iloc[i])
  total = total + hasil_akurasi
akurasi_semuanya = total/100
akurasi_semuanya

TypeError: ignored